In [1]:
import json
import logging
import requests
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Configure logging
logging.basicConfig(
    filename='app_errors.log',
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def get_sentiment_label(compound_score):
    if compound_score >= 0.05:
        return "positive"
    elif compound_score <= -0.05:
        return "negative"
    else:
        return "neutral"

def analyze_sentiment(text):
    try:
        if not text.strip():
            raise ValueError("Empty input provided for sentiment analysis.")

        # TextBlob sentiment
        blob = TextBlob(text)
        blob_sentiment = {
            'polarity': round(blob.sentiment.polarity, 3),
            'subjectivity': round(blob.sentiment.subjectivity, 3)
        }

        # VADER sentiment
        vader = SentimentIntensityAnalyzer()
        vader_scores = vader.polarity_scores(text)
        sentiment_label = get_sentiment_label(vader_scores['compound'])

        return {
            'input_text': text,
            'textblob': blob_sentiment,
            'vader': vader_scores,
            'sentiment_label': sentiment_label
        }
    except Exception as e:
        logging.error(f"Sentiment analysis failed: {e}")
        return {
            'error': 'Sentiment analysis failed.',
            'details': str(e)
        }

def fetch_random_quote():
    try:
        response = requests.get("https://zenquotes.io/api/random", timeout=5)
        response.raise_for_status()
        data = response.json()[0]
        return {
            'quote': data['q'],
            'author': data['a']
        }
    except requests.exceptions.RequestException as e:
        logging.error(f"Quote API request failed: {e}")
        return {
            'quote': 'Quote unavailable due to error.',
            'error': str(e)
        }

def mini_app(user_input):
    sentiment_data = analyze_sentiment(user_input)
    quote_data = fetch_random_quote()

    result = {
        'sentiment_analysis': sentiment_data,
        'inspirational_quote': quote_data
    }

    return json.dumps(result, indent=2)

if __name__ == "__main__":
    user_sentence = input("Enter a sentence: ")
    output_json = mini_app(user_sentence)
    print("\n--- Sentiment Summary ---\n")
    print(output_json)


Enter a sentence:  I love exploring new ideas. It's exciting.



--- Sentiment Summary ---

{
  "sentiment_analysis": {
    "input_text": "I love exploring new ideas. It's exciting.",
    "textblob": {
      "polarity": 0.312,
      "subjectivity": 0.618
    },
    "vader": {
      "neg": 0.0,
      "neu": 0.403,
      "pos": 0.597,
      "compound": 0.8126
    },
    "sentiment_label": "positive"
  },
  "inspirational_quote": {
    "quote": "Man suffers only because he takes seriously what the gods made for fun.",
    "author": "Alan Watts"
  }
}
